In [3]:
import json
import pandas as pd
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
with open("obligation_data/PBG & Retention money annotations.json", 'r', encoding="utf-8") as f:
    lines = json.load(f)

In [5]:
dataset = []
for line in lines:
    context = line['data']['clause_text']
    annotation = line['annotations']
    payment_terms = []
    all_labels=[]
    for annot in annotation:
        result = annot['result']
        for res in result:
            labels = res['value']['labels']      
            if labels[0] == "What is the amount for retention money?":
#                 print(res)
                value = res['value']
                start_id = value['start']
                end_id = value['end']
                payment_terms.append([start_id, end_id])
                all_labels.append(labels[0])
#             elif labels[0] == "What is the EMD amount?" and j<100:
#                 value = res['value']
#                 start_id = value['start']
#                 end_id = value['end']
#                 payment_terms.append([start_id, end_id])
#                 all_labels.append(labels[0])
#                 j+=1 
        if len(payment_terms) > 0:
            dataset.append([payment_terms, context, all_labels])
print(len(dataset))

33


In [6]:
print(dataset[0])

[[[6296, 7501]], '7 Performance Security\n7.1 Performance Security\n(i) (A) Within 30 (thirty) days of receipt of Letter of Acceptance, the selected Bidder\nshall furnish to the Authority an irrevocable and unconditional guarantee from a Bank\nin the form set forth in Annex-I of Schedule-G (the “Performance Security”) for an\namount equal to 3% (five percent) of its Bid Price. In case of bids mentioned below,\nthe Selected Bidder, along with the Performance Security, shall also furnish to the\nAuthority an irrevocable and unconditional guarantee from a Bank in the same form\ngiven at Annex-I of Schedule-G towards an Additional Performance Security (the\n“Additional Performance Security”) for an amount calculated as under:\n(a) If the Bid Price offered by the Selected Bidder is lower than 10% but upto\n20% of the estimated Project Cost, then the Additional Performance Security\nshall be calculated @20% of the difference in the (i) Estimated Project Cost\n(as mentioned in RFP)-10% of the

In [7]:
# splitting the json into words and labels. 
complete_data = []
#set of all words in our dataset
vocab = set()

all_context = []
#docanno answers
doccano_answers = []
low = []
words_context = []
x = []
y = []

#iterate over every data point (sentence - corpus of labelled text) to extract words
for data in dataset: 
    #list to hold all the labels
    labels = []
    #list to hold the extracted answer's class
    extracted_class = []
    #extract the text and annotations from the each data point
    text = data[1]
    indices = data[0]
    label = data[2]
    indices = sorted(indices)
    #print(ord_dict)
    words_context = text.split()             
    #print(words_context) 
    words_inside = []
    words_outside_3 = []
    all_words = []
    words_middle_entities = []
    label_check = []
    
    first = indices[0][0]
    last = indices[len(indices)-1][1]
    outside_entity1 = text[:first]
    words_oe1 = outside_entity1.split()

    outside_entity2 = text[last+1:]
    words_oe2 = outside_entity2.split()

    for i in range(len(indices)):
        start = indices[i][0]
        end = indices[i][1]
        inside_entity = text[start:end+1]
        words_ie = inside_entity.split()
        words_inside.append(words_ie)
#     print(words_inside)
#     print(len(words_inside), words_inside)
#     print(words_inside)
    #the outside entity in between the context.
    for j in range(len(indices)-1):
        end = indices[j+1][0] 
        start = indices[j][1]
        #print(start,end)
        outside_entity3 = text[start+1:end-1]
        words_oe3 = outside_entity3.split()
        words_outside_3.append(words_oe3)
#     print(len(words_outside_3), words_outside_3)
#     print(words_outside_3)
    label_oe1 = []
    label_oe2 = []
    label_oe3s = []
    label_ies = []
    label_middle_entities = []
    #print(words_inside)
    
    for i in range(len(words_oe1)):
        label_oe1.append('O')
    for i in range(len(words_oe2)):
        label_oe2.append('O')
        
    k = 0  
    for i in range(len(words_inside)):
        label_ie = []
        flag = 0
        for j in range(len(words_inside[i])):
            if label[i] == "What is the amount for retention money?":
                label_ie.append('I')
            else:
                label_ie.append('O')
        label_ies.append(label_ie)
        k+=1
    print(label_ies)
        
    for words_oe3 in words_outside_3:
        label_oe3 = []
        for i in range(len(words_oe3)):
            label_oe3.append('O')
        label_oe3s.append(label_oe3)
#     print(len(label_oe3s), len(label_ies))
    
    for i in range(len(label_oe3s)):
        label_middle_entities += label_ies[i]
        label_middle_entities += label_oe3s[i]
    label_middle_entities += label_ies[-1]
    
    for i in range(len(words_outside_3)):
        words_middle_entities += words_inside[i]
        words_middle_entities += words_outside_3[i]
    words_middle_entities += words_inside[-1]
    
    labels =  label_oe1 + label_middle_entities + label_oe2
    all_words = words_oe1 + words_middle_entities + words_oe2 
    #print(len(labels), len(all_words))
#     print(all_words)
    words_ies = " "
    for i in range(len(words_inside)):
        if 'I' in label_ies[i]:
            words_ies += " " + str(" ".join(words_inside[i]))

    if(len(labels)!=0):
        all_context.append(text)
        complete_data.append((words_context,labels))
        x.append(all_words)
        y.append(labels)
        doccano_answers.append(words_ies)
    else:
        x.append(all_words)
        y.append(['O']*len(words_context))
        doccano_answers.append([])

print(y)

[['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']]
[['I', 'I', 'I', 'I', 'I', 'I', 

In [8]:
print(len(doccano_answers))

33


In [9]:
contexts = [data[1] for data in dataset]
question = "What is the amount for retention money?"

In [10]:
print(len(contexts))

33


In [16]:
prompt = f"""
The following examples are contexts of a clause. The AI is required to read the context and perform extractive question answering.
Here are some examples:

context: {contexts[0]}
question: {question}
answer: {doccano_answers[0]}

context: {contexts[5]}
question: {question}
answer: {doccano_answers[5]}

context: {contexts[9]}
question: {question}
answers: 
"""

In [20]:
from langchain.llms import AI21
from langchain.chains.question_answering import load_qa_chain
AI21_API_KEY = "2dGQ0cOcc19E7HsAa77SbSfEGku8AylW"
llm = AI21(model="j2-jumbo-instruct", ai21_api_key=AI21_API_KEY, temperature=0)
#chain = load_qa_chain(llm, chain_type="stuff")

In [21]:
print(llm(prompt))

The Contractor shall obtain (at his cost) a Performance Security and an additional Performance Security, if any in terms of ITB 35.5, for proper performance of the contract, for the amount, currencies and validity period for Performance Security stated in the Contract Data. If an amount is not stated in the Contract Data, this Sub-Clause shall not apply.
The Contractor shall deliver the Performance Security and additional Performance Security, if any, to the Employer within 28 days after receiving the Letter of Acceptance, and shall send a copy to the Engineer. The Performance Security/additional Performance Security shall be issued by an entity and from within a country (or other jurisdiction) approved by the Employer, and shall be in the form as given in Section 8 (Contract Forms) or in another form specifically approved by the Employer.
The Performance Security/additional Performance Security shall be, at the Contractor's option, in any of the following forms: An unconditional Bank 

In [19]:
print(doccano_answers[9])

  Employer, on being satisfied with the claim of the contractor, shall return 75% of the amount of additional Performance Security against the particular schedule(s). The balance amount of additional Performance Security shall however be returned only after completion of the works against the particular schedule(s)/bill(s) for which additional Performance Security was submitted in terms of ITB 35.5, in all respects to the satisfaction of the Employer. Decision of the Employer regarding completion of works against a particular schedule/bill shall be final and binding on the contractor.


In [19]:
#examples for the few shot prompting
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate
examples = [
    {
        "context": contexts[0],
        "query": question,
        "answer": doccano_answers[0]
    },
    {
        "context": contexts[5],
        "query": question,
        "answer": doccano_answers[5]
    }
]

In [21]:
#example template of the prompt

example_template = """
Input: {context},
Input: {query},
Output: {answer}
"""

In [23]:
#creating the prompt example from the above template
example_prompt = PromptTemplate(
    input_variables=["context", "query", "answer"],
    template=example_template)

In [24]:
prefix = """The following examples are contexts of a clause. The AI is required to ingest the context and answer the question.
Here are some examples:
"""
suffix = """
Input: {context}
Input: {query}
AI: 
"""

In [25]:
#FEW SHOT PROMPT Template
few_shot_prompt_template = FewShotPromptTemplate(
        examples=examples,
        examples_prompt=example_prompt, 
        prefix=prefix,
        suffix=suffix,
        input_variables=["context", "query"],
        example_separator="\n\n")

ValidationError: 2 validation errors for FewShotPromptTemplate
example_prompt
  field required (type=value_error.missing)
examples_prompt
  extra fields not permitted (type=value_error.extra)